## Step 1: Setup and Installation

We install all required libraries for running the Phi-3-mini model and RAG pipeline:

- **API connection** (`huggingface_hub`)  
- **RAG and indexing** (`llama-index-core`, `llama-index-embeddings-huggingface`)  
- **Text embeddings** (`sentence-transformers`)  
- **PDF reading** (`pypdf`)  
- **PyTorch backend** (`torch`)  
- **Energy tracking** (`codecarbon`)  
- **TensorFlow/Keras backend support** (`tf-keras`)


In [ ]:
%pip install huggingface_hub
%pip install llama-index-core
%pip install llama-index-embeddings-huggingface
%pip install sentence-transformers
%pip install pypdf
%pip install torch
%pip install codecarbon
%pip install tf-keras

##  Step 2: Import Libraries
We import all libraries and configure:

- **Environment variables**  
- **API key**  
- **Path to Apollo dataset**  
- **Country code** for emissions tracking

Make sure to update `DATA_PATH` with the location of your Apollo documents.


In [ ]:
from google.colab import userdata


# Load API key and model from .env
HF_API_KEY = userdata.get("HF_TOKEN")
HF_MODEL_ID = userdata.get("HF_MODEL")

# this points to your the Apollo dataset
DATA_PATH = r"/content/sample_data/source_file.txt"

YOUR_COUNTRY_ISO_CODE = "EGY"

if not HF_API_KEY or not HF_MODEL_ID:
    raise ValueError(" HF_TOKEN or HF_MODEL not found. Check your .env file!")

print(f" Loaded HF API key and model: {HF_MODEL_ID}")
print(f" Data path: {DATA_PATH}")

 Loaded HF API key and model: Qwen/Qwen2.5-3B-Instruct
 Data path: /content/sample_data/source_file.txt


## Step 3 — Define Prompts

We define three prompt templates for our recursive refinement loop:

1. **Draft Prompt** — initial answer generation.  
2. **Critic Prompt** — evaluates draft for accuracy and relevance.  
3. **Refiner Prompt** — rewrites the draft based on critic feedback.

This allows for **structured, iterative improvement** of answers.


In [ ]:
DRAFT_SYSTEM_PROMPT = (
    "You are an expert assistant. Answer the user's question based *only* on the provided context. "
    "Do not mention the context or 'the text'. If the answer is not in the context, say so."
)
DRAFT_USER_TEMPLATE = "Context:\n{context_str}\n\nQuestion:\n{query_str}"

SUMMARIZATION_CRITIC_PROMPT = (
    "You are a 'Critic' AI. Evaluate the 'Draft Summary' based *only* on the 'Source Text'. "
    "Identify issues in Fidelity (does the summary correctly reflect the source?), "
    "Coverage (are key points included?), and Conciseness (is anything unnecessarily long or overly detailed?). "
    "Provide feedback as bullet points. If perfect, respond with 'The summary is perfect.'"
)

PARAPHRASING_CRITIC_PROMPT = (
    "You are a 'Critic' AI. Evaluate the 'Draft Paraphrase' based *only* on the 'Original Text'. "
    "Identify issues in Meaning Preservation (does it retain the original meaning?), "
    "Expression Quality (is it clear and natural?), and Originality (is it sufficiently reworded?). "
    "Provide feedback as bullet points. If perfect, respond with 'The paraphrase is perfect.'"
)

CREATIVE_CRITIC_PROMPT = (
    "You are a 'Critic' AI. Evaluate the 'Draft Creative Output' based *only* on the 'Writing Prompt'. "
    "Identify issues in Creativity (is it imaginative or fresh?), "
    "Prompt Alignment (does it follow the requested style, tone, or theme?), "
    "and Engagement (is it vivid, coherent, and compelling?). "
    "Provide feedback as bullet points. If perfect, respond with 'The creative piece is perfect.'"
)

CRITIC_USER_TEMPLATE = "Source Context:\n{context}\n\nOriginal Question:\n{question}\n\nDraft Answer:\n{draft}"

REFINER_SYSTEM_PROMPT = (
    "You are a 'Refiner' AI. Rewrite the 'Original Draft' to incorporate the 'Critic's Feedback'. "
    "Do not add new info. Do not mention the context, draft, or feedback. Just provide the final, improved answer."
)
REFINER_USER_TEMPLATE = "Original Draft:\n{draft}\n\nCritic's Feedback:\n{feedback}"

print(" Prompts defined!")

 Prompts defined!


## Step 4 — Initialize API and Index Data

- Connect to the Hugging Face API.  
- Create an **embedding model** for retrieval.  
- Load Apollo documents and build a **vector index**.  
- Prepare a **retriever** to fetch the most relevant document chunks.


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.embeddings import resolve_embed_model

# Embeddings for retrieval
embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

# Load documents
documents = SimpleDirectoryReader(input_files=[DATA_PATH]).load_data()

# Build vector index
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
retriever = index.as_retriever(similarity_top_k=3)

print(" Retriever is ready.")

 Retriever is ready.


## Step 5 — Helper Function to Call Hugging Face API

This function sends prompts to the API and returns the response.  

- Includes **retry logic** in case of temporary API failure.  
- Ensures that **all API calls are consistent** for draft, critic, and refiner steps.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer


def call_hf_api(system_prompt, user_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    retries = 3
    model_name = "Qwen/Qwen2.5-3B-Instruct"

    model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype="auto", device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    for attempt in range(retries):
        try:
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=2048,  # modity it to 2048
                temperature=0.1,
            )
            generated_ids = [
                output_ids[len(input_ids) :]
                for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]

            return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        except Exception as e:
            print(f"API Error (Attempt {attempt + 1}/{retries}): {e}")
            if attempt < retries - 1:
                time.sleep(10)

    return "Error: API failed."

## Step 6 — Recursive RAG Loop with CodeCarbon

- Ask a question about Apollo.  
- Retrieve relevant context using the retriever.  
- Generate an initial **draft answer**.  
- Run **refinement cycles**:
  - Critic evaluates draft
  - Refiner updates draft based on feedback
- Track **CPU energy and CO2 emissions** for each query.  
- Print final refined answer with **time and emission metrics**.


brainstorming:

Three specific tasks require imporvement where recusive editing may apply to it. These are: Summarizing, paraphrasing, and creatve_generation. I will create specific refinement prompts for these tasks based on specific requirements in which suit the task type.
If the task type matched these three, the prompt will go through the above function, else we will generate just a single answer with one iteration of thinking.

In [ ]:
import json
import time
import textwrap
from codecarbon import OfflineEmissionsTracker

# Load data
with open("/content/sample_data/test_data.json", "r") as file:
    data = json.load(file)

prompts = data["prompts"][17:]

CRITIC_SYSTEM_PROMPT = {
    "summarization": SUMMARIZATION_CRITIC_PROMPT,
    "paraphrasing": PARAPHRASING_CRITIC_PROMPT,
    "creative_generation": CREATIVE_CRITIC_PROMPT,
}

REFINEMENT_CYCLES = 3

tracker = OfflineEmissionsTracker(country_iso_code=YOUR_COUNTRY_ISO_CODE)
tracker.start()

print(f"\n--- Query Engine Ready (API-Powered, {REFINEMENT_CYCLES} cycles) ---")
print("Type 'exit' to quit.\n")

try:
    for prompt in prompts:
        query = prompt["prompt"]
        category = prompt["category"]

        tracker.start_task("Recursive RAG Query")
        start_time = time.time()

        retrieved_nodes = retriever.retrieve(query)
        context_str = "\n---\n".join([node.get_content() for node in retrieved_nodes])

        # Generate draft
        user_prompt = DRAFT_USER_TEMPLATE.format(
            context_str=context_str, query_str=query
        )

        # Creating the draft (1st iteration of text gerneration).
        draft_text = call_hf_api(DRAFT_SYSTEM_PROMPT, user_prompt)
        print(f"\n📝 Initial Draft:\n{category}\n", textwrap.fill(draft_text, width=80))

        # Skip refinement if category does not match
        if category not in ["summarization", "paraphrasing", "creative_generation"]:
            print("✅ Draft approved early!")
            continue

        current_draft = draft_text

        for i in range(REFINEMENT_CYCLES):
            print(f"\n🔁 Refinement Cycle {i + 1}/{REFINEMENT_CYCLES}")

            critic_prompt = CRITIC_USER_TEMPLATE.format(
                context=context_str, question=query, draft=current_draft
            )

            feedback = call_hf_api(CRITIC_SYSTEM_PROMPT[category], critic_prompt)
            print("\n🧩 Critic Feedback:\n", feedback)

            refiner_prompt = REFINER_USER_TEMPLATE.format(
                draft=current_draft, feedback=feedback
            )

            current_draft = call_hf_api(REFINER_SYSTEM_PROMPT, refiner_prompt)
            print("\n✏️ Refined Draft:\n", textwrap.fill(current_draft, width=80))

        final_answer = current_draft
        end_time = time.time()
        emissions_data = tracker.stop_task()

        print("\n✅ Final Answer:\n", textwrap.fill(final_answer, width=80))
        print(f"\n--- Query Metrics ---\nTime: {end_time - start_time:.2f}s")
        print(f"Emissions: {emissions_data.emissions * 1000:.6f} gCO2eq")

finally:
    total_emissions = tracker.stop()
    print("\n🌍 Total Emissions Summary:")
    print(f"Total Local CO2: {total_emissions * 1000:.6f} gCO2eq")
    print("Report saved as 'emissions.csv'")

[codecarbon INFO @ 19:21:31] offline tracker init
[codecarbon WARNING @ 19:21:31] Multiple instances of codecarbon are allowed to run at the same time.



--- Query Engine Ready (API-Powered, 3 cycles) ---
Type 'exit' to quit.



config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


🧩 Critic Feedback:
 - The draft lacks creativity and imagination, failing to explore the broader implications and societal impacts of the Moon landing.
- It strictly follows the factual account without engaging in speculative or imaginative storytelling.
- The narrative is not vivid, coherent, or compelling, as it fails to paint a picture of the potential changes and reactions to the Moon landing on Earth.
- The draft misses the opportunity to engage readers by providing a narrative that delves into the psychological, technological, and cultural shifts that might have occurred following such a significant event.
- To improve, the draft should include vivid descriptions of how people's lives might have changed, such as advancements in space exploration, changes in education and science, and shifts in public opinion and government policies.
- Speculative elements should be incorporated to imagine how the Moon landing might have influenced various aspects of daily life, such as technolog

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


✏️ Refined Draft:
 The draft lacks creativity and imagination, failing to explore the broader
implications and societal impacts of the Moon landing. It strictly follows the
factual account without engaging in speculative or imaginative storytelling. The
narrative is not vivid, coherent, or compelling, as it fails to paint a picture
of the potential changes and reactions to the Moon landing on Earth. The draft
misses the opportunity to engage readers by providing a narrative that delves
into the psychological, technological, and cultural shifts that might have
occurred following such a significant event.  To improve, the draft should
include vivid descriptions of how people's lives might have changed, such as
advancements in space exploration, changes in education and science, and shifts
in public opinion and government policies. Speculative elements should be
incorporated to imagine how the Moon landing might have influenced various
aspects of daily life, such as technological innovat

/usr/local/lib/python3.12/dist-packages/codecarbon/output_methods/file.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)
